In [1]:
import geopandas as gpd
import pandas as pd
import verde as vd
import numpy as np
from shapely import geometry
import my_funcs as mf

import matplotlib.pyplot as plt
%matplotlib widget

gdb='/home/ggrl/geodatabase/'

In [89]:
# DADOS DE FLIGHT LINES E TIES CONCATENADOS

# m_scrr_1105 = pd.read_csv(gdb+'geof/recortado/m_scrr_1105')

g_scrr_1105 = pd.read_csv(gdb+'geof/recortado/g_scrr_1105')

In [3]:
# DADO SEM LINHAS DE AMARRAÇÃO

gama_cols = 'KB DATA BARO UB THB COSMICO CTB UUP ALTURA KPERC eU eTH CTEXP UTHRAZAO X Y UKRAZAO MDT THKRAZAO LIVE_TIME CTCOR KCOR THCOR UCOR HORA GPSALT LATITUDE FIDUCIAL TEMP LONGITUDE'.split(" ")


g_1105 = pd.read_csv(gdb+'xyz/1105_XYZ/1105_GamaLine.XYZ',
                         names=gama_cols,
                         delim_whitespace=True,
                         skiprows=11,
                         usecols=["X","Y","LATITUDE","LONGITUDE",
                                  "KPERC","eU","eTH","CTCOR","MDT"])

g_1105 = g_1105.dropna(inplace=False)

In [ ]:
region = [-46.75, -46.50,
          -22.50, -22.25]

g_scrr_1105 = g_1105[vd.inside((g_1105.LONGITUDE, g_1105.LATITUDE), region = region)]

In [4]:
g_scrr_1105

,KPERC,eU,eTH,X,Y,MDT,CTCOR,LATITUDE,LONGITUDE
24712,3.131,3.010,21.450,319876.46,7538416.79,852.63,13.168,-22.250395,-46.747921
24713,3.138,2.694,19.762,319877.01,7538335.07,854.22,12.603,-22.251133,-46.747925
24714,3.195,2.584,24.965,319877.79,7538253.58,855.56,12.501,-22.251869,-46.747927
24715,2.956,3.048,26.830,319878.80,7538172.18,853.89,12.673,-22.252604,-46.747926
24716,2.605,2.967,27.796,319879.77,7538090.60,847.30,12.911,-22.253341,-46.747926
...,...,...,...,...,...,...,...,...,...
95699,-0.051,0.911,8.642,345353.87,7538382.05,881.91,2.979,-22.253180,-46.500762
95700,0.139,0.860,8.537,345354.70,7538462.38,877.62,3.368,-22.252454,-46.500746
95701,0.232,1.269,10.269,345355.65,7538542.77,874.30,3.516,-22.251728,-46.500729
95702,0.311,1.515,10.976,345356.45,7538623.12,874.49,3.675,-22.251003,-46.500714


In [5]:
chain_list = ['CTCOR', 'KPERC', 'MDT', 'eU', 'eTH']

grids = {'CTCOR':(),'KPERC':(), 'MDT':(), 'eU':(), 'eTH':()}

coordinates = (g_scrr_1105.X.values, g_scrr_1105.Y.values)

for i in chain_list:
    print(i)
    
    chain = vd.Chain([
        ('trend', vd.Trend(degree=1)),
        ('reduce', vd.BlockReduce(np.median, spacing=499)),
        ('spline', vd.Spline())
    ])
    
    chain.fit(coordinates, g_scrr_1105[i])
    
    grid = chain.grid(spacing=100, data_names=[i],pixel_register=True)
    grids[i] = vd.distance_mask(coordinates, maxdist=499, grid= grid)

CTCOR
KPERC
MDT
eU
eTH


In [6]:
data = list()
for i in chain_list:
    df = grids[i].to_dataframe()
    data.append(df[i])

In [7]:
geof = pd.concat(data,axis=1, join='inner')

In [8]:
geof.reset_index(inplace=True)
geof

,easting,northing,CTCOR,KPERC,MDT,eU,eTH
0,319887.886745,7.510855e+06,4.038868,1.396738,707.889585,1.367782,11.311061
1,319887.886745,7.510955e+06,4.000219,1.363911,707.250762,1.406168,11.219762
2,319887.886745,7.511055e+06,3.955838,1.334353,706.902784,1.432748,11.114221
3,319887.886745,7.511155e+06,3.906529,1.309119,706.881479,1.444532,11.009487
4,319887.886745,7.511255e+06,3.853342,1.289124,707.255929,1.439711,10.931332
...,...,...,...,...,...,...,...
71140,345331.953255,7.538254e+06,3.629228,0.163444,877.561853,1.219581,9.544362
71141,345331.953255,7.538354e+06,3.527640,0.116976,879.402452,1.237646,9.271982
71142,345331.953255,7.538454e+06,3.417787,0.076915,881.916823,1.255049,9.000585
71143,345331.953255,7.538554e+06,3.299956,0.049595,885.264247,1.261531,8.744498


In [9]:
geof['geometry'] = [geometry.Point(x,y) for x, y in zip(geof['easting'], geof['northing'])]

In [10]:
gdf = gpd.GeoDataFrame(geof,crs=32723)
gdf = gdf.set_crs(32723, allow_override=True)

In [11]:
gdf

,easting,northing,CTCOR,KPERC,MDT,eU,eTH,geometry
0,319887.886745,7.510855e+06,4.038868,1.396738,707.889585,1.367782,11.311061,POINT (319887.887 7510854.938)
1,319887.886745,7.510955e+06,4.000219,1.363911,707.250762,1.406168,11.219762,POINT (319887.887 7510954.933)
2,319887.886745,7.511055e+06,3.955838,1.334353,706.902784,1.432748,11.114221,POINT (319887.887 7511054.928)
3,319887.886745,7.511155e+06,3.906529,1.309119,706.881479,1.444532,11.009487,POINT (319887.887 7511154.923)
4,319887.886745,7.511255e+06,3.853342,1.289124,707.255929,1.439711,10.931332,POINT (319887.887 7511254.918)
...,...,...,...,...,...,...,...,...
71140,345331.953255,7.538254e+06,3.629228,0.163444,877.561853,1.219581,9.544362,POINT (345331.953 7538253.592)
71141,345331.953255,7.538354e+06,3.527640,0.116976,879.402452,1.237646,9.271982,POINT (345331.953 7538353.587)
71142,345331.953255,7.538454e+06,3.417787,0.076915,881.916823,1.255049,9.000585,POINT (345331.953 7538453.582)
71143,345331.953255,7.538554e+06,3.299956,0.049595,885.264247,1.261531,8.744498,POINT (345331.953 7538553.577)


In [12]:
gdf.crs

<Projected CRS: EPSG:32723>
Name: WGS 84 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World - S hemisphere - 48°W to 42°W - by country
- bounds: (-48.0, -80.0, -42.0, 0.0)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [14]:
plt.figure()
grids['MDT'].MDT.plot(cmap='terrain')
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(319837.80000000005, 345382.04, 7510804.94, 7538703.57)

In [15]:
plt.figure()
grids['CTCOR'].CTCOR.plot(cmap='hsv')
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(319837.80000000005, 345382.04, 7510804.94, 7538703.57)

In [78]:
vmin=geof.KPERC.quantile(0.1)
vmax=geof.KPERC.quantile(0.99)
plt.figure()
grids['KPERC'].KPERC.plot(cmap='hsv',vmin=vmin,vmax=vmax)
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(319837.80000000005, 345382.04, 7510804.94, 7538703.57)

In [17]:
plt.figure()
grids['eTH'].eTH.plot(cmap='hsv')
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(319837.80000000005, 345382.04, 7510804.94, 7538703.57)

In [51]:
vmin=df_eU.eU.min()
vmax=df_eU.eU.max()
plt.figure()
grids['eU'].eU.plot(cmap='hsv',vmin=vmin,vmax=vmax)
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(319837.80000000005, 345382.04, 7510804.94, 7538703.57)

In [32]:
df_eU = grids['eU'].to_dataframe()

In [44]:
df_eU = df_eU.reset_index()

In [73]:
print(geof.eU.min())

-0.46645587408708167


In [76]:
print(geof.KPERC.min())
print(geof.KPERC.quantile([0.10,0.5,0.90]))

-0.19870656642770257
0.1    0.310994
0.5    1.009870
0.9    1.991911
Name: KPERC, dtype: float64


In [5]:
socorro = gpd.read_file(gdb+'litostrat/250k/socorro.shp')
socorro = socorro.drop('id',axis=1)

In [82]:
socorro.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [176]:
geof['closest_unid'] = gdf['geometry'].apply(lambda x: socorro['litologia'].iloc[socorro.distance(x).idxmin()]) # .idxmin() Retorna o indice do menor valor 

In [177]:
geof

,easting,northing,CTCOR,KPERC,MDT,eU,eTH,geometry,closest_unid
0,319887.886745,7.510855e+06,4.038868,1.396738,707.889585,1.367782,11.311061,POINT (319887.887 7510854.938),47
1,319887.886745,7.510955e+06,4.000219,1.363911,707.250762,1.406168,11.219762,POINT (319887.887 7510954.933),47
2,319887.886745,7.511055e+06,3.955838,1.334353,706.902784,1.432748,11.114221,POINT (319887.887 7511054.928),47
3,319887.886745,7.511155e+06,3.906529,1.309119,706.881479,1.444532,11.009487,POINT (319887.887 7511154.923),47
4,319887.886745,7.511255e+06,3.853342,1.289124,707.255929,1.439711,10.931332,POINT (319887.887 7511254.918),47
...,...,...,...,...,...,...,...,...,...
71140,345331.953255,7.538254e+06,3.629228,0.163444,877.561853,1.219581,9.544362,POINT (345331.953 7538253.592),37
71141,345331.953255,7.538354e+06,3.527640,0.116976,879.402452,1.237646,9.271982,POINT (345331.953 7538353.587),37
71142,345331.953255,7.538454e+06,3.417787,0.076915,881.916823,1.255049,9.000585,POINT (345331.953 7538453.582),37
71143,345331.953255,7.538554e+06,3.299956,0.049595,885.264247,1.261531,8.744498,POINT (345331.953 7538553.577),37


In [186]:
type(geof.closest_unid[0])

str

In [184]:
geof.to_csv(gdb+'orange_input_socorro.csv',index=False)

In [1]:
# DADOS DO AEROLEVANTAMENTO 1039 - FLIGHT LINES 1km

In [2]:
geof_socorro_cut = pd.read_csv(gdb+'geof/recortado/geof_socorro_cut.csv')

In [3]:
geof_socorro_cut

,UTME,UTMN,LONG,LAT,MAGR,THC,UC,KC,CTC,geometry
0,243279.0,7497977.0,-47.497177,-22.605408,113.40,78.97,28.26,10.75,1214.07,POINT (243279 7497977)
1,243277.0,7498075.0,-47.497177,-22.604523,111.35,77.53,18.51,32.60,1188.08,POINT (243277 7498075)
2,243275.0,7498174.0,-47.497192,-22.603622,108.29,77.11,0.00,74.30,1269.47,POINT (243275 7498174)
3,243272.0,7498272.0,-47.497192,-22.602737,105.23,74.49,0.54,15.39,1265.21,POINT (243272 7498272)
4,243270.0,7498370.0,-47.497192,-22.601852,97.17,56.10,6.14,33.21,1090.33,POINT (243270 7498370)
...,...,...,...,...,...,...,...,...,...,...
180859,329329.0,7563776.0,-46.653473,-22.022064,92.53,70.13,20.86,245.36,2177.98,POINT (329329 7563776)
180860,329426.0,7563774.0,-46.652527,-22.022095,100.51,54.54,19.79,279.66,2144.92,POINT (329426 7563774)
180861,329523.0,7563771.0,-46.651596,-22.022125,100.50,74.22,19.75,236.94,2081.99,POINT (329523 7563771)
180862,329620.0,7563769.0,-46.650650,-22.022156,96.48,68.61,5.81,235.28,1978.74,POINT (329620 7563769)


In [6]:
socorro_250k = mf.litologia('socorro_250k')

In [8]:
socorro_250k.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [14]:
region = [300000,350000,
          7400000,7513000]

In [15]:
geof_socorro_cut = geof_socorro_cut[vd.inside((geof_socorro_cut.UTME, geof_socorro_cut.UTMN), region = region)]

In [16]:
coordinates = (geof_socorro_cut.UTME.values, geof_socorro_cut.UTMN.values)

# Block Reduction
reducer_median = vd.BlockReduce(np.mean, spacing= 1000)
b_coords, b_CTC = reducer_median.filter(coordinates, geof_socorro_cut.CTC)


# Ploting the Reduced data
plt.figure(figsize=(11,11))
plt.scatter(b_coords[0],b_coords[1],
            c    = b_CTC,
            cmap = 'hsv',
            s    = 2)
plt.axis('scaled')
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
geof_socorro_cut

,UTME,UTMN,LONG,LAT,MAGR,THC,UC,KC,CTC,geometry
93981,300340.0,7400023.0,-46.955139,-23.497406,41.22,66.40,11.99,76.19,1411.93,POINT (300340 7400023)
93982,300337.0,7400122.0,-46.955170,-23.496521,51.18,79.33,17.08,97.39,1399.23,POINT (300337 7400122)
93983,300333.0,7400221.0,-46.955185,-23.495621,61.14,64.35,0.00,87.21,1249.43,POINT (300333 7400221)
93984,300329.0,7400321.0,-46.955215,-23.494720,69.11,53.52,21.21,71.54,1207.55,POINT (300329 7400321)
93985,300326.0,7400420.0,-46.955231,-23.493820,76.07,74.26,20.44,30.65,1208.15,POINT (300326 7400420)
...,...,...,...,...,...,...,...,...,...,...
175297,344823.0,7504431.0,-46.509155,-22.559448,393.64,81.34,9.73,13.51,1200.66,POINT (344823 7504431)
175298,344924.0,7504432.0,-46.508163,-22.559448,364.62,121.82,6.40,0.12,1405.71,POINT (344924 7504432)
175299,345025.0,7504433.0,-46.507187,-22.559448,327.59,130.11,11.86,45.08,1551.80,POINT (345025 7504433)
175300,345124.0,7504434.0,-46.506226,-22.559448,297.56,127.84,0.60,30.97,1539.77,POINT (345124 7504434)


In [45]:
chain_list = ['THC', 'UC', 'KC', 'CTC', 'MAGR']

grids = {'THC':(),'UC':(), 'KC':(), 'CTC':(), 'MAGR':()}

scores = {'THC':(),'UC':(), 'KC':(), 'CTC':(), 'MAGR':()}


coordinates = (geof_socorro_cut.UTME.values, geof_socorro_cut.UTMN.values)

for i in chain_list:
    print(i)
    
    chain = vd.Chain([
        ('trend', vd.Trend(degree=1)),
        ('reduce', vd.BlockReduce(np.median, spacing=3000)),
        ('spline', vd.Spline())
    ])
    
    chain.fit(coordinates, geof_socorro_cut[i])

    cv     = vd.BlockKFold(spacing=3000,
                  n_splits=25,
                  shuffle=True)

    scores[i] = vd.cross_val_score(chain,
                             coordinates,
                             geof_socorro_cut[i],
                             cv=cv)
    
    grid = chain.grid(spacing=600, data_names=[i],pixel_register=True)
    grids[i] = vd.distance_mask(coordinates, maxdist=3000, grid= grid)


THC
UC
KC
CTC
MAGR


In [46]:
scores

{'THC': array([0.53670398, 0.40951402, 0.63001902, 0.38880332, 0.4907072 ,
        0.55584648, 0.55902851, 0.55593506, 0.58659222, 0.52600309,
        0.5736337 , 0.64696113, 0.64002141, 0.53340164, 0.59105131,
        0.53634521, 0.58168415, 0.67116536, 0.37096197, 0.49908998,
        0.56981871, 0.60097155, 0.50610175, 0.59331178, 0.63792719]),
 'UC': array([-0.02950878,  0.22258222,  0.18724895,  0.13833292,  0.19473033,
         0.31613688,  0.15191187,  0.24659781,  0.29700907,  0.08089541,
         0.25913108,  0.25398818,  0.04640495,  0.28426917,  0.02430159,
         0.17884278,  0.08354211,  0.17835265,  0.22446672,  0.36078904,
         0.22416785,  0.31735679,  0.11178309,  0.16124318,  0.10852914]),
 'KC': array([0.33272182, 0.41267712, 0.50976389, 0.22286122, 0.36041077,
        0.24457321, 0.51494736, 0.27899553, 0.41641912, 0.45955427,
        0.35977059, 0.39421836, 0.36776729, 0.15589062, 0.30570735,
        0.00205792, 0.44837874, 0.3333265 , 0.29214549, 0.18449763,


In [48]:
plt.figure()
plt.hist(scores['THC'],bins='auto')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([3., 0., 2., 5., 6., 4., 5.]),
 array([0.37096197, 0.41384817, 0.45673437, 0.49962057, 0.54250677,
        0.58539296, 0.62827916, 0.67116536]),
 <BarContainer object of 7 artists>)

In [21]:
plt.figure()
grids['CTC'].CTC.plot(cmap='hsv',vmin=0)
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(300000.0, 346378.0, 7400006.0, 7513000.0)

In [29]:
plt.figure()
grids['THC'].THC.plot(cmap='hsv',vmin=0)
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(300000.0, 346378.0, 7400006.0, 7513000.0)

In [27]:
plt.figure()
grids['KC'].KC.plot(cmap='hsv',vmin=0)
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(300000.0, 346378.0, 7400006.0, 7513000.0)

In [26]:
plt.figure()
grids['UC'].UC.plot(cmap='hsv',vmin=0)
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(300000.0, 346378.0, 7400006.0, 7513000.0)

In [30]:
plt.figure()
grids['MAGR'].MAGR.plot(cmap='hsv')
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(300000.0, 346378.0, 7400006.0, 7513000.0)

In [31]:
grids

{'THC': <xarray.Dataset>
 Dimensions:   (easting: 232, northing: 565)
 Coordinates:
   * easting   (easting) float64 3.001e+05 3.003e+05 ... 3.461e+05 3.463e+05
   * northing  (northing) float64 7.4e+06 7.4e+06 ... 7.513e+06 7.513e+06
 Data variables:
     THC       (northing, easting) float64 66.21 66.99 67.72 ... nan nan nan
 Attributes:
     metadata:  Generated by Chain(steps=[('trend', Trend(degree=1)),\n       ...,
 'UC': <xarray.Dataset>
 Dimensions:   (easting: 232, northing: 565)
 Coordinates:
   * easting   (easting) float64 3.001e+05 3.003e+05 ... 3.461e+05 3.463e+05
   * northing  (northing) float64 7.4e+06 7.4e+06 ... 7.513e+06 7.513e+06
 Data variables:
     UC        (northing, easting) float64 16.59 16.82 17.08 ... nan nan nan
 Attributes:
     metadata:  Generated by Chain(steps=[('trend', Trend(degree=1)),\n       ...,
 'KC': <xarray.Dataset>
 Dimensions:   (easting: 232, northing: 565)
 Coordinates:
   * easting   (easting) float64 3.001e+05 3.003e+05 ... 3.461e+05 3